In [2]:
from cifar10 import  cifar10_input
import tensorflow as tf
import numpy as np

In [3]:
batch_size = 128
print("begin")
images_train, labels_train = cifar10_input.inputs(eval_data = False, batch_size = batch_size)
images_test, labels_test = cifar10_input.inputs(eval_data = True, batch_size = batch_size)
print("begin data")

begin
begin data


In [4]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)
  
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  
                        
def avg_pool_6x6(x):
    return tf.nn.avg_pool(x, ksize=[1, 6, 6, 1], strides=[1, 6, 6, 1], padding='SAME')

In [5]:
# tf Graph Input
x = tf.placeholder(tf.float32, [None, 24,24,3]) # mnist data维度 28*28=784
y = tf.placeholder(tf.float32, [None, 10]) # 0-9 数字=> 10 classes

In [6]:
W_conv1 = weight_variable([5, 5, 3, 64])
b_conv1 = bias_variable([64])

x_image = tf.reshape(x, [-1,24,24,3])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 64, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

#############################################全连接
keep_prob = tf.placeholder("float")
W_fc1 = weight_variable([6 * 6 * 64, 256])
b_fc1 = bias_variable([256])

h_pool2_flat = tf.reshape(h_pool2, [-1, 6*6*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([256, 128])
b_fc2 = bias_variable([128])
h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

W_fc3 = weight_variable([128, 10])
b_fc3 = bias_variable([10])
y_conv=tf.nn.softmax(tf.matmul(h_fc2, W_fc3) + b_fc3)


cross_entropy = -tf.reduce_sum(y*tf.log(y_conv))

#不同的优化方法测测效果
#train_step = tf.train.GradientDescentOptimizer(1e-3).minimize(cross_entropy)
#train_step = tf.train.AdagradOptimizer(1e-5).minimize(cross_entropy)
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [7]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# tf.train.start_queue_runners(sess=sess)
for i in range(15000):#20000
    image_batch, label_batch = sess.run([images_train, labels_train])
    label_b = np.eye(10,dtype=float)[label_batch] #one hot
  
    train_step.run(feed_dict={x:image_batch, y: label_b, keep_prob: 0.5},session=sess)
  
    if i%200 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:image_batch, y: label_b, keep_prob: 1.0},session=sess)
        print( "step %d, training accuracy %g"%(i, train_accuracy))


image_batch, label_batch = sess.run([images_test, labels_test])
label_b = np.eye(10,dtype=float)[label_batch]#one hot
print ("finished！ test accuracy %g"%accuracy.eval(feed_dict={x:image_batch, y: label_b, keep_prob: 1.0},session=sess))

step 0, training accuracy 0.0859375
step 200, training accuracy 0.28125
step 400, training accuracy 0.351562
step 600, training accuracy 0.421875
step 800, training accuracy 0.40625
step 1000, training accuracy 0.34375
step 1200, training accuracy 0.398438
step 1400, training accuracy 0.53125
step 1600, training accuracy 0.5
step 1800, training accuracy 0.515625
step 2000, training accuracy 0.453125
step 2200, training accuracy 0.5
step 2400, training accuracy 0.53125
step 2600, training accuracy 0.570312
step 2800, training accuracy 0.570312
step 3000, training accuracy 0.5
step 3200, training accuracy 0.5625
step 3400, training accuracy 0.59375
step 3600, training accuracy 0.570312
step 3800, training accuracy 0.5625
step 4000, training accuracy 0.554688
step 4200, training accuracy 0.484375
step 4400, training accuracy 0.523438
step 4600, training accuracy 0.601562
step 4800, training accuracy 0.609375
step 5000, training accuracy 0.5
step 5200, training accuracy 0.632812
step 5400,